# 문제 연설문을 기계학습하기 위해 변환

## pyspark 기본 셋팅

In [1]:
import os
import pyspark

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

### DataFrame 생성

In [2]:
conRdd=spark.sparkContext\
    .textFile(os.path.join("data","ds_congratulations.txt")).map(lambda x: (x,))
conDf = spark.createDataFrame(conRdd, ["sent"])
conDf.show()

+----------------------------------+
|                              sent|
+----------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|
|                                  |
| 국민의 안전을 위해 밤낮없이 애...|
|                                  |
|오늘 홍조근정훈장을 받으신 중앙...|
|                                  |
|           사랑하는 경찰관 여러분,|
|                                  |
|여러분의 헌신적 노력으로 우리의...|
|                                  |
| 치안의 개선은 국민의 체감으로 ...|
|                                  |
| 한국을 찾는 외국 관광객들도 우...|
|                                  |
|   올해는 ‘경찰의 날’에 맞춰 국...|
|                                  |
|         자랑스러운 경찰관 여러분,|
|                                  |
| 경찰헌장은 “나라와 겨레를 위하...|
|                                  |
+----------------------------------+
only showing top 20 rows



### 단어로 분리해서, 출력

In [3]:
from pyspark.ml.feature import RegexTokenizer
TK = RegexTokenizer(inputCol="sent", outputCol="wordsReg", pattern="\\s+")
conDf=TK.transform(conDf)
conDf.show()

+----------------------------------+--------------------------------+
|                              sent|                        wordsReg|
+----------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|
|                                  |                              []|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|
|                                  |                              []|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|
|                                  |                              []|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|
|                                  |                              []|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...|
|                                  |                              []|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|
|                                  |                              []|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 관광객...|
|                                  |     

### 불용어 구성, 출력 - 축사 전문에서 한 단어를 스스로 구성

In [4]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="wordsReg", outputCol="nostops")

stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

_mystopwords=[u"경찰관",u"여러분"]

for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)

for e in stop.getStopWords():
    print (e, end="/")

i/me/my/myself/we/our/ours/ourselves/you/your/yours/yourself/yourselves/he/him/his/himself/she/her/hers/herself/it/its/itself/they/them/their/theirs/themselves/what/which/who/whom/this/that/these/those/am/is/are/was/were/be/been/being/have/has/had/having/do/does/did/doing/a/an/the/and/but/if/or/because/as/until/while/of/at/by/for/with/about/against/between/into/through/during/before/after/above/below/to/from/up/down/in/out/on/off/over/under/again/further/then/once/here/there/when/where/why/how/all/any/both/each/few/more/most/other/some/such/no/nor/not/only/own/same/so/than/too/very/s/t/can/will/just/don/should/now/i'll/you'll/he'll/she'll/we'll/they'll/i'd/you'd/he'd/she'd/we'd/they'd/i'm/you're/he's/she's/it's/we're/they're/i've/we've/you've/they've/isn't/aren't/wasn't/weren't/haven't/hasn't/hadn't/don't/doesn't/didn't/won't/wouldn't/shan't/shouldn't/mustn't/can't/couldn't/cannot/could/here's/how's/let's/ought/that's/there's/what's/when's/where's/who's/why's/would/경찰관/여러분/

### 불용어 제거하고, 출력

In [5]:
stopDf=stop.transform(conDf)
stopDf.show()

+----------------------------------+--------------------------------+--------------------------------+
|                              sent|                        wordsReg|                         nostops|
+----------------------------------+--------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|   [존경하는, 국민, 여러분,, ...|
|                                  |                              []|                              []|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|  [국민의, 안전을, 위해, 밤낮...|
|                                  |                              []|                              []|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 홍조근정훈장을, 받으신...|
|                                  |                              []|                              []|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|             [사랑하는, 여러분,]|
|                                  |                              []|                             

###  TF-IDF를 계산하고, 출력

In [6]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashDf = hashTF.transform(stopDf)

idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)
idfDf.select("nostops", "hash").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|nostops                                                                                                                                                                                                                                                                 |hash                                                                                                                                                                                 

### TF-IDF 컬럼을 features로 구성, 출력

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

feat = VectorAssembler(inputCols=["hash","idf"],outputCol="features")
featDf = feat.transform(idfDf)
featDf.show(5)

+----------------------------------+--------------------------------+--------------------------------+--------------------+--------------------+--------------------+
|                              sent|                        wordsReg|                         nostops|                hash|                 idf|            features|
+----------------------------------+--------------------------------+--------------------------------+--------------------+--------------------+--------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|   [존경하는, 국민, 여러분,, ...|(262144,[162,8073...|(262144,[162,8073...|(524288,[162,8073...|
|                                  |                              []|                              []|      (262144,[],[])|      (262144,[],[])|      (524288,[],[])|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|  [국민의, 안전을, 위해, 밤낮...|(262144,[5341,630...|(262144,[5341,630...|(524288,[5341,630...|
|                                  |                              [